# Random Forest - Tania/Kenny

In [1]:
#!python --version


In [2]:
#install libraries
!pip install scikeras
!pip install np_utils
!pip install graphviz
!pip install pydotplus


#import dependencies and other ML libraries
import tensorflow as tf
from keras.utils import to_categorical
from scikeras.wrappers import KerasClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score, roc_curve, auc, roc_auc_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV, learning_curve, cross_val_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, label_binarize
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

# Commonly used supporting libraries
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix
import seaborn as sns

from sklearn.ensemble import RandomForestClassifier



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
#Load Waterfowl Dataset

csv="ShortTermSetData(Aug-Sept).csv"

#Read data
#dataset=pd.read_csv(csv)
dataset = pd.read_csv(csv, on_bad_lines='skip', engine='python')
dataset.head()

,event-id,visible,timestamp,location-long,location-lat,acceleration-raw-x,acceleration-raw-y,acceleration-raw-z,bar:barometric-height,battery-charge-percent,...,mag:magnetic-field-raw-x,mag:magnetic-field-raw-y,mag:magnetic-field-raw-z,orn:transmission-protocol,tag-voltage,sensor-type,individual-taxon-canonical-name,tag-local-identifier,individual-local-identifier,study-name
0,34849161874,True,2024-08-15 00:09:28.000,-91.142067,40.723488,169.0,-11.0,1052.0,0.0,100,...,0.611,-0.594,-0.452,GPRS,4178.0,gps,Anas platyrhynchos,223151,WLR-52-M-YR1,Mallard Connectivity
1,34849161875,True,2024-08-15 01:09:52.000,-91.141777,40.722813,81.0,222.0,994.0,0.0,100,...,0.582,-0.241,-0.451,GPRS,4173.0,gps,Anas platyrhynchos,223151,WLR-52-M-YR1,Mallard Connectivity
2,34849161876,True,2024-08-15 02:09:44.000,-91.141586,40.722385,95.0,232.0,951.0,0.0,100,...,0.770,-0.415,-0.403,GPRS,4170.0,gps,Anas platyrhynchos,223151,WLR-52-M-YR1,Mallard Connectivity
3,34849161877,True,2024-08-15 03:09:22.000,-91.141655,40.722443,114.0,219.0,1001.0,0.0,100,...,0.492,-0.697,-0.450,GPRS,4170.0,gps,Anas platyrhynchos,223151,WLR-52-M-YR1,Mallard Connectivity
4,34849161878,True,2024-08-15 04:09:44.000,-91.141670,40.722382,185.0,-236.0,1016.0,0.0,100,...,0.093,-0.367,-0.421,GPRS,4167.0,gps,Anas platyrhynchos,223151,WLR-52-M-YR1,Mallard Connectivity


In [4]:
#Cleaning up rows and columns
dataset.shape

#Showing descriptive statistics
dataset.describe()

,event-id,location-long,location-lat,acceleration-raw-x,acceleration-raw-y,acceleration-raw-z,bar:barometric-height,battery-charge-percent,battery-charging-current,external-temperature,...,gps-time-to-fix,ground-speed,heading,height-above-msl,gls:light-level,mag:magnetic-field-raw-x,mag:magnetic-field-raw-y,mag:magnetic-field-raw-z,tag-voltage,tag-local-identifier
count,7.779300e+04,77793.000000,77793.000000,77793.000000,77793.000000,77793.000000,77793.0,77793.000000,77793.000000,77793.000000,...,77793.00000,77793.000000,77793.000000,77793.000000,77793.000000,77793.000000,77793.000000,77793.00000,77793.000000,77793.000000
mean,3.539654e+10,-99.906410,48.976472,-54.162277,60.955562,949.234803,0.0,97.386654,1.850321,30.725271,...,22.37593,0.154241,153.027805,457.105845,767.128649,0.207779,0.348341,-0.53750,4152.488720,233297.122042
std,4.676999e+08,6.275237,4.453480,240.485468,234.638435,142.504095,0.0,8.499285,3.849531,6.203482,...,15.45548,1.051803,119.653738,222.362742,514.110854,1.290925,0.912754,1.90012,56.717171,4282.394490
min,3.484422e+10,-123.232689,36.051205,-1727.000000,-1703.000000,-1352.000000,0.0,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.000000,-1866.000000,0.000000,-2.022000,-1.911000,-16.99200,3542.000000,223122.000000
25%,3.508480e+10,-103.100304,46.724056,-202.000000,-60.000000,927.000000,0.0,100.000000,0.000000,27.000000,...,9.44000,0.000000,36.000000,345.000000,97.000000,-0.177000,-0.024000,-0.69000,4157.000000,234439.000000
50%,3.525923e+10,-100.035942,48.870514,-61.000000,100.000000,975.000000,0.0,100.000000,0.000000,30.000000,...,17.23000,0.000000,144.000000,493.000000,1049.000000,0.071000,0.320000,-0.40300,4170.000000,234522.000000
75%,3.545121e+10,-98.315132,51.140259,77.000000,228.000000,1008.000000,0.0,100.000000,2.000000,34.000000,...,32.16000,0.000000,263.000000,571.000000,1108.000000,0.342000,0.626000,-0.11200,4178.000000,236820.000000
max,3.718302e+10,-80.690582,66.027733,1388.000000,1593.000000,1838.000000,0.0,100.000000,28.000000,59.000000,...,91.45000,36.111111,360.000000,9004.000000,2047.000000,12.195000,7.746000,1.85800,4191.000000,236889.000000


In [5]:
#Identifying data types and names of each column
dataset.dtypes

event-id                             int64
visible                               bool
timestamp                           object
location-long                      float64
location-lat                       float64
acceleration-raw-x                 float64
acceleration-raw-y                 float64
acceleration-raw-z                 float64
bar:barometric-height              float64
battery-charge-percent               int64
battery-charging-current           float64
external-temperature               float64
gps:hdop                           float64
gps:satellite-count                  int64
gps-time-to-fix                    float64
ground-speed                       float64
heading                            float64
height-above-msl                   float64
import-marked-outlier                 bool
gls:light-level                    float64
mag:magnetic-field-raw-x           float64
mag:magnetic-field-raw-y           float64
mag:magnetic-field-raw-z           float64
orn:transmi

In [6]:
#Checking for any null values
dataset.isnull().sum()

event-id                           0
visible                            0
timestamp                          0
location-long                      0
location-lat                       0
acceleration-raw-x                 0
acceleration-raw-y                 0
acceleration-raw-z                 0
bar:barometric-height              0
battery-charge-percent             0
battery-charging-current           0
external-temperature               0
gps:hdop                           0
gps:satellite-count                0
gps-time-to-fix                    0
ground-speed                       0
heading                            0
height-above-msl                   0
import-marked-outlier              0
gls:light-level                    0
mag:magnetic-field-raw-x           0
mag:magnetic-field-raw-y           0
mag:magnetic-field-raw-z           0
orn:transmission-protocol          0
tag-voltage                        0
sensor-type                        0
individual-taxon-canonical-name    0
t

In [7]:
#Checking out data distribution
for x in dataset.columns:
  print(f"Column: {x}")
  print("-"*20)
  print(dataset[x].value_counts())
  print("")

Column: event-id
--------------------
event-id
35450297093    1
35450297077    1
35450297076    1
35450297075    1
35450297074    1
              ..
34849161879    1
34849161878    1
34849161877    1
34849161876    1
34849161875    1
Name: count, Length: 77793, dtype: int64

Column: visible
--------------------
visible
True    77793
Name: count, dtype: int64

Column: timestamp
--------------------
timestamp
2024-09-09 16:06:05.000    3
2024-09-18 11:26:23.000    3
2024-08-30 03:52:30.000    3
2024-08-19 08:52:05.000    3
2024-08-28 13:52:43.000    3
                          ..
2024-09-19 03:34:56.000    1
2024-09-19 04:35:07.000    1
2024-09-19 05:35:16.000    1
2024-09-19 06:35:29.000    1
2024-08-15 06:09:20.000    1
Name: count, Length: 76343, dtype: int64

Column: location-long
--------------------
location-long
-85.623940     22
-90.245758     21
-100.797073    19
-90.245735     19
-98.087585     18
               ..
-111.815643     1
-111.816177     1
-111.815895     1
-111.8178

In [8]:
# Drop irrelevant columns and verify, only if they exist
columns_to_drop = ["battery-charge-percent", "battery-charging-current", "gps-time-to-fix",
                   "orn:transmission-protocol", "tag-voltage", "sensor-type", "visible", "individual-local-identifier"]

for column in columns_to_drop:
    if column in dataset.columns:  # Check if column exists before dropping
        dataset = dataset.drop(columns=[column])
    # else:
    #     print(f"Column '{column}' not found in the DataFrame.") Print a message if the column is not found

dataset.columns #verification

Index(['event-id', 'timestamp', 'location-long', 'location-lat',
       'acceleration-raw-x', 'acceleration-raw-y', 'acceleration-raw-z',
       'bar:barometric-height', 'external-temperature', 'gps:hdop',
       'gps:satellite-count', 'ground-speed', 'heading', 'height-above-msl',
       'import-marked-outlier', 'gls:light-level', 'mag:magnetic-field-raw-x',
       'mag:magnetic-field-raw-y', 'mag:magnetic-field-raw-z',
       'individual-taxon-canonical-name', 'tag-local-identifier',
       'study-name'],
      dtype='object')

In [9]:

# Convert timestamp column to datetime and then to Unix timestamp (seconds since epoch)
dataset['timestamp'] = pd.to_datetime(dataset['timestamp'], errors='coerce')
dataset['timestamp'] = dataset['timestamp'].astype('int64') // 10**9  # Converts to seconds

# Define column types
numeric_cols = ['event-id', 'location-long', 'location-lat', 'acceleration-raw-x', 'acceleration-raw-y', 'acceleration-raw-z', 
                'bar:barometric-height', 'external-temperature', 'gps:hdop', 'heading', 'gps:satellite-count', 'ground-speed', 'height-above-msl', 'gls:light-level', 
                'mag:magnetic-field-raw-x', 'mag:magnetic-field-raw-y', 'mag:magnetic-field-raw-z', 'tag-local-identifier']

categorical_cols = ['import-marked-outlier']  # Add other categorical columns
string_cols = ['individual-taxon-canonical-name', 'study-name']

# Apply transformations
for col in dataset.columns:
    if col in string_cols:
        dataset[col] = dataset[col].astype(str)  # Preserve as string
    elif col in categorical_cols:
        dataset[col], _ = pd.factorize(dataset[col])  # Encode categorical
    elif col in numeric_cols:
        dataset[col] = pd.to_numeric(dataset[col], errors='coerce')  # Ensure numeric values stay numeric
    else:
        dataset[col] = dataset[col]  # Default behavior

# One-Hot Encoding for Species Column
one_hot_encoder = OneHotEncoder(sparse_output=False)
species_encoded = one_hot_encoder.fit_transform(dataset[['individual-taxon-canonical-name']])

# Convert to DataFrame with meaningful column names
species_encoded_df = pd.DataFrame(species_encoded, columns=one_hot_encoder.get_feature_names_out())

# Concatenate with dataset and drop original species column
dataset = pd.concat([dataset, species_encoded_df], axis=1).drop(columns=['individual-taxon-canonical-name'])

# One-Hot Encoding for the 'study-name' Column
one_hot_encoder = OneHotEncoder(sparse_output=False)
study_name_encoded = one_hot_encoder.fit_transform(dataset[['study-name']])

# Convert to DataFrame with meaningful column names
study_name_encoded_df = pd.DataFrame(study_name_encoded, columns=one_hot_encoder.get_feature_names_out(['study-name']))

# Concatenate with the original dataset and drop the original 'study-name' column
dataset = pd.concat([dataset, study_name_encoded_df], axis=1).drop(columns=['study-name'])

# Check results
pd.options.display.max_columns = 50
dataset.head()


,event-id,timestamp,location-long,location-lat,acceleration-raw-x,acceleration-raw-y,acceleration-raw-z,bar:barometric-height,external-temperature,gps:hdop,gps:satellite-count,ground-speed,heading,height-above-msl,import-marked-outlier,gls:light-level,mag:magnetic-field-raw-x,mag:magnetic-field-raw-y,mag:magnetic-field-raw-z,tag-local-identifier,individual-taxon-canonical-name_Anas crecca,individual-taxon-canonical-name_Anas platyrhynchos,individual-taxon-canonical-name_Anas rubripes,individual-taxon-canonical-name_Anas strepera,study-name_Mallard Connectivity
0,34849161874,1723680568,-91.142067,40.723488,169.0,-11.0,1052.0,0.0,32.0,1.8,4,0.000000,84.0,159.0,0,754.0,0.611,-0.594,-0.452,223151,0.0,1.0,0.0,0.0,1.0
1,34849161875,1723684192,-91.141777,40.722813,81.0,222.0,994.0,0.0,33.0,1.5,6,1.944444,347.0,180.0,0,79.0,0.582,-0.241,-0.451,223151,0.0,1.0,0.0,0.0,1.0
2,34849161876,1723687784,-91.141586,40.722385,95.0,232.0,951.0,0.0,32.0,1.3,6,0.000000,311.0,165.0,0,91.0,0.770,-0.415,-0.403,223151,0.0,1.0,0.0,0.0,1.0
3,34849161877,1723691362,-91.141655,40.722443,114.0,219.0,1001.0,0.0,31.0,1.8,5,0.000000,328.0,166.0,0,87.0,0.492,-0.697,-0.450,223151,0.0,1.0,0.0,0.0,1.0
4,34849161878,1723694984,-91.141670,40.722382,185.0,-236.0,1016.0,0.0,29.0,1.3,6,0.000000,261.0,158.0,0,83.0,0.093,-0.367,-0.421,223151,0.0,1.0,0.0,0.0,1.0


In [10]:
#Algorithm to start splitting between training and test sets

# Labels
y = dataset['event-id']
# Features
X = dataset.drop(columns = 'event-id')

#Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

#Split training data into training and validation
X_train, X_validate, y_train, y_validate = train_test_split(X_train, y_train, random_state=1)

In [11]:

# Convert to numpy arrays
X_train = X_train.to_numpy()
X_validate = X_validate.to_numpy()
X_test = X_test.to_numpy()


y_train = y_train.to_numpy()
y_validate = y_validate.to_numpy()
y_test = y_test.to_numpy()

In [12]:
#Checking shape of the new datasets

print(f"Shape of X_train: {X_train.shape}")
print(f"Shape of X_test: {X_test.shape}")
print(f"Shape of X_validate: {X_validate.shape}")

print(f"Shape of y_train: {y_train.shape}")
print(f"Shape of y_test: {y_test.shape}")
print(f"Shape of y_validate: {y_validate.shape}")

Shape of X_train: (43758, 24)
Shape of X_test: (19449, 24)
Shape of X_validate: (14586, 24)
Shape of y_train: (43758,)
Shape of y_test: (19449,)
Shape of y_validate: (14586,)


In [13]:
train_class_counts = {label: (y_train == label).sum() for label in np.unique(y_train)}
test_class_counts = {label: (y_test == label).sum() for label in np.unique(y_test)}

print(f"Distribution of y_train: {train_class_counts}")
print(f"Distribution of y_test: {test_class_counts}")

Distribution of y_train: {np.int64(34844219717): np.int64(1), np.int64(34844785298): np.int64(1), np.int64(34844785299): np.int64(1), np.int64(34844785301): np.int64(1), np.int64(34844785302): np.int64(1), np.int64(34845731867): np.int64(1), np.int64(34845731868): np.int64(1), np.int64(34846107563): np.int64(1), np.int64(34846107564): np.int64(1), np.int64(34846107565): np.int64(1), np.int64(34846107567): np.int64(1), np.int64(34846107568): np.int64(1), np.int64(34846107569): np.int64(1), np.int64(34846107579): np.int64(1), np.int64(34846626641): np.int64(1), np.int64(34846626642): np.int64(1), np.int64(34846626644): np.int64(1), np.int64(34846626645): np.int64(1), np.int64(34846626648): np.int64(1), np.int64(34847936715): np.int64(1), np.int64(34847936717): np.int64(1), np.int64(34847936718): np.int64(1), np.int64(34847936722): np.int64(1), np.int64(34847936723): np.int64(1), np.int64(34847936724): np.int64(1), np.int64(34847936725): np.int64(1), np.int64(34848033341): np.int64(1), np

In [14]:
def visualize_first_tree(model):
    from sklearn.tree import plot_tree
    plt.figure(figsize=(20,10))
    plot_tree(model.estimators_[0], filled=True, feature_names=X_train.columns, class_names=True, rounded=True, fontsize=8)
    plt.show()

In [15]:

def plot_learning_curve(model, title, X, y):
    train_sizes, train_scores, test_scores = learning_curve(model, X, y, cv=2, n_jobs=-1, train_sizes=np.linspace(0.1, 1.0, 5))
    
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    
    fig, ax = plt.subplots(figsize=(8, 6))
    ax.set_title(title)
    ax.set_xlabel("Training examples")
    ax.set_ylabel("Score")
    ax.grid()

    ax.fill_between(train_sizes, train_scores_mean - train_scores_std,
                    train_scores_mean + train_scores_std, alpha=0.1, color="r")
    ax.fill_between(train_sizes, test_scores_mean - test_scores_std,
                    test_scores_mean + test_scores_std, alpha=0.1, color="purple")

    ax.plot(train_sizes, train_scores_mean, 'o-', color='r', label="Training score")
    ax.plot(train_sizes, test_scores_mean, 'o-', color='purple', label="Cross-validation score")

    ax.legend(loc="best")
    
    plt.show()  
    return fig  # Return figure if needed for further manipulation


In [16]:
from sklearn.preprocessing import label_binarize
from sklearn.metrics import roc_auc_score

def plot_roc_curve(model, X, y, ax=None):
    """
    Plots the ROC curve for a trained model. Works for both binary and multiclass classification.
    """
    # Check if the model supports probability predictions
    if hasattr(model, "predict_proba"):
        y_pred_proba = model.predict_proba(X)  # Get class probabilities
    else:
        raise ValueError("Model does not support probability outputs.")

    # Binarize y labels for multiclass ROC handling
    y_bin = label_binarize(y, classes=np.unique(y))
    
    # Compute ROC AUC score for multiclass
    roc_auc = roc_auc_score(y_bin, y_pred_proba, average='macro', multi_class='ovr')

    # Plot the ROC Curve
    if ax is None:
        plt.figure()
        ax = plt.gca()
    
    ax.set_title(f"ROC Curve (AUC = {roc_auc:.2f})")
    ax.set_xlabel("False Positive Rate")
    ax.set_ylabel("True Positive Rate")
    ax.plot([0,1], [0,1], 'k--')  # Diagonal reference line
    ax.legend(loc="lower right")

    print(f"ROC-AUC Score: {roc_auc:.4f}")
    
    return ax


In [17]:
# Loop over each model in an array of models
def evaluate_model(model, X_train, y_train, X_test, y_test, name):
    """
    Trains and evaluates a model using accuracy, F1-score, ROC curve, learning curve, and confusion matrix.

    Parameters:
    model -- The machine learning model to evaluate.
    X_train, y_train -- Training dataset.
    X_test, y_test -- Test dataset.
    name -- Name of the model for labeling.

    Returns:
    None
    """
    
    # Fit the model
    model.fit(X_train, y_train)

    # Make predictions
    y_pred = model.predict(X_test)

    # Evaluate Accuracy and F1 Score
    accuracy = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average='macro')

    # Print Results
    print(f"Model: {name}, Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}")

    # Create Subplots
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))
    fig.suptitle(f"{name} - Accuracy: {accuracy:.4f}, F1 Score: {f1:.4f}")

    # Learning Curve (Check if function exists)
    if 'plotLearningCurve' in globals():
        plotLearningCurve(model, f"Learning Curve - {name}", X_train, y_train, ax=axes[0])
    else:
        axes[0].set_title("Learning Curve Not Available")
        axes[0].axis("off")

    # ROC Curve (Handle unsupported models)
    try:
        plot_roc_curve(model, X_test, y_test, ax=axes[1])
    except ValueError as e:
        axes[1].set_title("ROC Curve Not Available")
        axes[1].axis("off")
        print(f"Warning: Could not plot ROC curve for {name}. Error: {e}")

    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[2])
    axes[2].set_title("Confusion Matrix")
    axes[2].set_xlabel("Predicted Labels")
    axes[2].set_ylabel("True Labels")

    plt.show()
    print("\n")

    # Print Classification Report
    print("Classification Report:")
    print(classification_report(y_test, y_pred))

In [18]:
# Initializes an array called models.
models = []
models.append(('FOREST', RandomForestClassifier()))

In [ ]:
# Testing Train and evaluate using only the first 100 samples
for name, model in models:
    print(f"Training {name} with first 100 entries...")
    model.fit(X_train[:100], y_train[:100])  # Train only on first 100 samples
    evaluate_model(model, X_train[:100], y_train[:100], X_test, y_test, name)

# Visualize the first tree
visualize_first_tree(models[0][1])  # Assuming the first model in the list is the RandomForestClassifier

Training FOREST with first 100 entries...
Model: FOREST, Accuracy: 0.0000, F1 Score: 0.0000


C:\Users\atomi\AppData\Local\Temp\ipykernel_20724\1274445102.py:29: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  ax.legend(loc="lower right")


ROC-AUC Score: 0.4722


In [ ]:
# Train and evaluate using the full dataset
for name, model in models:
    print(f"Training {name} with the full dataset...")
    model.fit(X_train, y_train)  # Train on the entire dataset
    evaluate_model(model, X_train, y_train, X_test, y_test, name)

In [ ]:
visualize_first_tree(models[0][1])  # Assuming the first model in the list is the RandomForestClassifier

In [ ]:
param_dist = {
    'n_estimators': [100, 200, 300, 400, 500],  # Number of trees
    'max_depth': [None, 10, 20, 30, 40, 50],     # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],              # Minimum samples to split a node
    'min_samples_leaf': [1, 2, 4],               # Minimum samples at a leaf node
    'max_features': ['sqrt', 'log2', None],       # Number of features to consider for splits
    'bootstrap': [True, False]                    # Whether to use bootstrap samples
}

In [ ]:

# Initialize the base model
rf = RandomForestClassifier(random_state=42)

# Perform RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=50,  # Number of parameter settings to sample
    cv=3,       # Number of cross-validation folds
    scoring='accuracy',  # Metric to optimize (can also use 'f1', 'roc_auc', etc.)
    n_jobs=-1,  # Use all available CPU cores
    random_state=42,
    verbose=2
)

# Fit the model
random_search.fit(X_train, y_train)

# Print the best parameters and best score
print("Best Parameters:", random_search.best_params_)
print("Best Cross-Validation Score:", random_search.best_score_)

In [ ]:
# Train the final model with the best parameters
best_rf = random_search.best_estimator_

# Evaluate the tuned model
evaluate_model(best_rf, X_train, y_train, X_test, y_test, "Tuned Random Forest")

In [ ]:
# Initial model
initial_rf = RandomForestClassifier(random_state=42)
evaluate_model(initial_rf, X_train, y_train, X_test, y_test, "Initial Random Forest")

# Tuned model
evaluate_model(best_rf, X_train, y_train, X_test, y_test, "Tuned Random Forest")

import joblib

# Save the tuned model
joblib.dump(best_rf, 'tuned_random_forest_duck_flight_path.pkl')